# Prevendo os sobreviventes do Titanic
# Modelo 4

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [2]:
treino = pd.read_csv(r'C:\Users\Lenovo\Documents\MeusProjetos\Data-Science\Titanic\train.csv')
teste = pd.read_csv(r'C:\Users\Lenovo\Documents\MeusProjetos\Data-Science\Titanic\test.csv')

treino.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [3]:
treino.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Concatenando, transformando e separando treino e teste

In [4]:
#Adicionar coluna com a origem
treino['Origem'] = 'treino'
teste['Origem'] = 'teste'

treino.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Origem
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,treino
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,treino
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,treino
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,treino
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,treino


In [5]:
# Concatenando os DataFrames
df_concatenado = pd.concat([treino, teste], ignore_index=True)
df_concatenado

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Origem
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,treino
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,treino
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,treino
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,treino
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,treino
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,1305,NaN,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S,teste
1305,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,teste
1306,1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,teste
1307,1308,NaN,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S,teste


### Transformações

In [6]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df_concatenado['Embarked'] = label_encoder.fit_transform(df_concatenado['Embarked'])
df_concatenado['Sex'] = label_encoder.fit_transform(df_concatenado['Sex'])

df_concatenado.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Origem
0,1,0.0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,2,treino
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,0,treino
2,3,1.0,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,2,treino
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,2,treino
4,5,0.0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,2,treino


In [7]:
print(df_concatenado.isna().sum())

PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          0
Origem            0
dtype: int64


In [8]:
df_concatenado['Age'].fillna(np.mean(df_concatenado['Age']), inplace=True)
df_concatenado['Fare'].fillna(np.mean(df_concatenado['Fare']), inplace=True)
df_concatenado['Embarked'].fillna(-1, inplace=True)

### Separando os dataframes

In [9]:
df_concatenado.shape

(1309, 13)

In [10]:
# Separando os DataFrames com base na coluna de origem
treino = df_concatenado[df_concatenado['Origem'] == 'treino'].drop(columns=['Origem'])
teste = df_concatenado[df_concatenado['Origem'] == 'teste'].drop(columns=['Origem', 'Survived'])

treino.shape

(891, 12)

In [11]:
teste.shape

(418, 11)

In [12]:
teste.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
891,892,3,"Kelly, Mr. James",1,34.5,0,0,330911,7.8292,NaN,1
892,893,3,"Wilkes, Mrs. James (Ellen Needs)",0,47.0,1,0,363272,7.0000,NaN,2
893,894,2,"Myles, Mr. Thomas Francis",1,62.0,0,0,240276,9.6875,NaN,1
894,895,3,"Wirz, Mr. Albert",1,27.0,0,0,315154,8.6625,NaN,2
895,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",0,22.0,1,1,3101298,12.2875,NaN,2


In [13]:
print(teste.isna().sum())
print()
print(treino.isna().sum())

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
dtype: int64

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64


## Selecionando as features

In [14]:
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
target = 'Survived'

In [15]:
from sklearn.model_selection import train_test_split
X_treino, X_valid, y_treino, y_valid = train_test_split(treino[features], treino[target], random_state=10, test_size=0.5)

## Verificando parâmetros para a DecisionTreeClassifier

In [16]:
def get_acc_DecisionTree(Xtrain, Xvalid, ytrain, yvalid, SamplesLeaf=[10], LeafNodes=[4], RandomState=0):
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.metrics import accuracy_score
    
    maior_acc_samples = 0
    melhor_sample = SamplesLeaf[0]
    for samples in SamplesLeaf:
        modelo = DecisionTreeClassifier(random_state=RandomState, min_samples_leaf=samples)
        modelo.fit(Xtrain, ytrain)
        ypred = modelo.predict(Xvalid)
        accuracy = accuracy_score(ypred, yvalid)
        if accuracy > maior_acc_samples:
            maior_acc_samples = accuracy
            melhor_sample = samples

    maior_acc_nodes = 0
    melhor_nodes = LeafNodes[0]

    for nodes in LeafNodes:
        modelo = DecisionTreeClassifier(random_state=RandomState, max_leaf_nodes=nodes)
        modelo.fit(Xtrain, ytrain)
        ypred = modelo.predict(Xvalid)
        accuracy = accuracy_score(ypred, yvalid)
        if accuracy > maior_acc_nodes:
            maior_acc_nodes = accuracy
            melhor_nodes = nodes

    print(f'Melhor min_samples_leaf: {melhor_sample} com {maior_acc_samples:.4f} de acurácia')
    print(f'Melhor max_leaf_nodes: {melhor_nodes} com {maior_acc_nodes:.4f} de acurácia')


In [17]:
get_acc_DecisionTree(X_treino, X_valid, y_treino, y_valid, SamplesLeaf=list(range(2,20, 2)), LeafNodes=list(range(2, 400, 2)))

Melhor min_samples_leaf: 14 com 0.8251 de acurácia
Melhor max_leaf_nodes: 8 com 0.8318 de acurácia


## Treinando o modelo4 com max_leaf_nodes = 8

In [18]:
X_treino, y_treino = treino[features], treino[target]
X_teste = teste[features]

modelo4 = DecisionTreeClassifier(random_state=0, max_leaf_nodes=8)
modelo4.fit(X_treino, y_treino)
y_pred4 = modelo4.predict(X_teste)

y_pred4

array([0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0.,
       0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0.,
       0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0.,
       0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0.,
       0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0.,
       1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 1.,
       0., 0., 0., 0., 0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 1., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0., 1.,
       0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1.,
       0., 1., 0., 1., 1.

In [19]:
#Estava em float
y_pred4 = y_pred4.astype(int)
y_pred4

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

## Arquivo de submissão

In [20]:
sub = pd.Series(y_pred4, index=teste['PassengerId'], name='Survived')
sub.shape

(418,)

In [21]:
sub.to_csv('Quarto_modelo.csv', header=True)

# Modelo 5 - Random Forests

In [22]:
X_treino, X_valid, y_treino, y_valid = train_test_split(treino[features], treino[target], random_state=0, test_size=0.5)

modelo5 = RandomForestClassifier(n_estimators=100, n_jobs=-1, max_leaf_nodes=8, random_state=0)
modelo5.fit(X_treino, y_treino)
y_pred = modelo5.predict(X_valid)
acc = accuracy_score(y_valid, y_pred)
acc

0.804932735426009

In [23]:
def get_acc_RandomForest(Xtrain, Xvalid, ytrain, yvalid, SamplesLeaf=[10], LeafNodes=[4], estimators=[100],RandomState=0):
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import accuracy_score
    
    maior_acc_samples = 0
    melhor_sample = SamplesLeaf[0]
    for samples in SamplesLeaf:
        modelo = RandomForestClassifier(random_state=RandomState, min_samples_leaf=samples, n_jobs=-1)
        modelo.fit(Xtrain, ytrain)
        ypred = modelo.predict(Xvalid)
        accuracy = accuracy_score(ypred, yvalid)
        if accuracy > maior_acc_samples:
            maior_acc_samples = accuracy
            melhor_sample = samples

    maior_acc_nodes = 0
    melhor_nodes = LeafNodes[0]

    for nodes in LeafNodes:
        modelo = RandomForestClassifier(random_state=RandomState, max_leaf_nodes=nodes, n_jobs=-1)
        modelo.fit(Xtrain, ytrain)
        ypred = modelo.predict(Xvalid)
        accuracy = accuracy_score(ypred, yvalid)
        if accuracy > maior_acc_nodes:
            maior_acc_nodes = accuracy
            melhor_nodes = nodes

    maior_acc_estimators = 0
    melhor_estimators = estimators[0]

    for est in estimators:
        modelo = RandomForestClassifier(random_state=RandomState, n_estimators=est, n_jobs=-1)
        modelo.fit(Xtrain, ytrain)
        ypred = modelo.predict(Xvalid)
        accuracy = accuracy_score(ypred, yvalid)
        if accuracy > maior_acc_estimators:
            maior_acc_estimators = accuracy
            melhor_estimators = est

    print(f'Melhor min_samples_leaf: {melhor_sample} com {maior_acc_samples:.4f} de acurácia')
    print(f'Melhor max_leaf_nodes: {melhor_nodes} com {maior_acc_nodes:.4f} de acurácia')
    print(f'Melhor n_estimators: {melhor_estimators} com {maior_acc_estimators:.4f} de acurácia')

In [24]:
get_acc_RandomForest(X_treino, X_valid, y_treino, y_valid, SamplesLeaf=list(range(2,20, 2)), LeafNodes=list(range(2, 100, 2)), estimators=list(range(40, 241, 40)))

Melhor min_samples_leaf: 6 com 0.8161 de acurácia
Melhor max_leaf_nodes: 30 com 0.8206 de acurácia
Melhor n_estimators: 40 com 0.8004 de acurácia


In [25]:
#validação 1
modelo5 = RandomForestClassifier(random_state=0, max_leaf_nodes=30, n_estimators=40, n_jobs=-1)
modelo5.fit(X_treino, y_treino)
ypred = modelo5.predict(X_valid)
accuracy = accuracy_score(ypred, y_valid)
accuracy

0.8004484304932735

In [26]:
#validação 2
modelo5 = RandomForestClassifier(random_state=0, max_leaf_nodes=30, n_estimators=150, n_jobs=-1)
modelo5.fit(X_treino, y_treino)
ypred = modelo5.predict(X_valid)
accuracy = accuracy_score(ypred, y_valid)
accuracy

0.8094170403587444

In [27]:
#validação 3
modelo5 = RandomForestClassifier(random_state=0, n_estimators=150, min_samples_leaf=6, n_jobs=-1)
modelo5.fit(X_treino, y_treino)
ypred = modelo5.predict(X_valid)
accuracy = accuracy_score(ypred, y_valid)
accuracy

0.8026905829596412

In [28]:
#validação 4
modelo5 = RandomForestClassifier(random_state=0, max_leaf_nodes=30, n_jobs=-1)
modelo5.fit(X_treino, y_treino)
ypred = modelo5.predict(X_valid)
accuracy = accuracy_score(ypred, y_valid)
accuracy

0.820627802690583

## Treinando o modelo 5 com a validação 4

In [29]:
X_treino, y_treino = treino[features], treino[target]
X_teste = teste[features]

modelo5 = RandomForestClassifier(random_state=0, max_leaf_nodes=30, n_jobs=-1)
modelo5.fit(X_treino, y_treino)
y_pred5 = modelo5.predict(X_teste)

y_pred5 = y_pred5.astype(int)

y_pred5

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

## Arquivo de submissão

In [30]:
def get_submission(pred, index='PassengerId', name='Survived', name_file='file.csv'):
    import pandas as pd
    sub = pd.Series(pred, index=teste[index], name=name)
    sub.to_csv(name_file, header=True)

In [31]:
get_submission(y_pred5, name_file='Quinto_modelo.csv')

# Modelo 6 - Regressão Logística

In [32]:
from sklearn.linear_model import LogisticRegression

In [33]:
def get_acc_LogisticRegression(train_df, size_test=0.4, features_list=[], target_name='target' ,iterations=[100, 300, 500, 700, 1000], solvers=['lbfgs', 'newton-cg', 'sag', 'saga']):
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import accuracy_score

    X_train, X_valid, y_train, y_valid = train_test_split(train_df[features_list], train_df[target_name], random_state=42, test_size=size_test)

    maior_acc_iter_solver = 0
    melhor_iteration = iterations[0]
    melhor_solver = solvers[0]
    for it in iterations:
        for solv in solvers:
            model = LogisticRegression(random_state=42, solver=solv, max_iter=it, n_jobs=-1)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_valid)
            acc = accuracy_score(y_valid, y_pred)
            if acc >= maior_acc_iter_solver:
                maior_acc_iter_solver = acc
                melhor_iteration = it
                melhor_solver = solv

    print(f'Melhor iteração: {melhor_iteration}')
    print(f'Melhor solver: {melhor_solver}')
    print(f'Acurácia: {maior_acc_iter_solver}')

### Normalizando os dados

In [34]:
def normalizar_colunas_numericas(df, numeric_cols=[]):
    '''
    df: DataFrame com colunas numéricas
    numeric_cols (list): Lista com nome das colunas numéricas
    '''
    from sklearn.preprocessing import MinMaxScaler
    normalizador = MinMaxScaler()
    for col in numeric_cols:
        df[col] = normalizador.fit_transform(df[[col]])

    return df

In [35]:
treino = normalizar_colunas_numericas(treino, ['Age', 'Fare'])
treino.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",1,0.271174,1,0,A/5 21171,0.014151,NaN,2
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,0.472229,1,0,PC 17599,0.139136,C85,0
2,3,1.0,3,"Heikkinen, Miss. Laina",0,0.321438,0,0,STON/O2. 3101282,0.015469,NaN,2
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,0.434531,1,0,113803,0.103644,C123,2
4,5,0.0,3,"Allen, Mr. William Henry",1,0.434531,0,0,373450,0.015713,NaN,2


In [36]:
get_acc_LogisticRegression(train_df=treino, features_list=features, target_name=target)

c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Melhor iteração: 1000
Melhor solver: saga
Acurácia: 0.7899159663865546


In [37]:
get_acc_LogisticRegression(train_df=treino, features_list=features, target_name=target, size_test=0.5)

c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Melhor iteração: 1000
Melhor solver: saga
Acurácia: 0.7937219730941704


In [38]:
get_acc_LogisticRegression(train_df=treino, features_list=features, target_name=target, size_test=0.3)

c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Melhor iteração: 1000
Melhor solver: saga
Acurácia: 0.8097014925373134


## Treinando o modelo 6 com max_iter=1000 e solver='saga'

In [39]:
X_treino, y_treino = treino[features], treino[target]
X_teste = teste[features]

modelo6 = LogisticRegression(random_state=42, max_iter=1000, solver='saga', n_jobs=-1)
modelo6.fit(X_treino, y_treino)

y_pred6 = modelo6.predict(X_teste)

y_pred6 = y_pred6.astype(int)

y_pred6

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

## Arquivo de submissão 6

In [40]:
get_submission(y_pred6, name_file='Sexto_modelo.csv')

# Modelo 7 - XGBoost

In [41]:
from skopt import gp_minimize
from xgboost import XGBClassifier

## Criando uma função para maximizar a acurácia

In [42]:
X_treino, X_valid, y_treino, y_valid = train_test_split(treino[features], treino[target], random_state=0, test_size=0.5)

In [47]:
def treinar_xgb_acc(params):
    learning_rate = params[0]
    max_depth = params[1]
    min_child_weight = params[2]
    subsample = params[3]
    colsample_bytree = params[4]

    modelo = XGBClassifier(
        learning_rate=learning_rate,
        max_depth=max_depth,
        min_child_weight=min_child_weight,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        random_state=0,
        n_estimators=100)

    modelo.fit(X_treino, y_treino)
    pred = modelo.predict(X_valid)

    return -accuracy_score(pred, y_valid)    #negativo pois a função de otimização é de minimização

space = [(0.001, 0.5),  #learning_rate
         (2, 128),  #num_leaves
         (1, 100),  #min_child_samples
         (0.05, 1.0),  #subsample
         (0.11, 1.0)]  #colsample_bytree    

## Otimizando os parâmetros com Bayesian Optimization

In [48]:
resultados_gp = gp_minimize(treinar_xgb_acc, space, random_state=0, verbose=1, n_calls=100, n_random_starts=10)

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.0921
Function value obtained: -0.6345
Current minimum: -0.6345
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0903
Function value obtained: -0.7668
Current minimum: -0.7668
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.0908
Function value obtained: -0.7018
Current minimum: -0.7668
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.0760
Function value obtained: -0.6345
Current minimum: -0.7668
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 0.0648
Function value obtained: -0.6345
Current minimum: -0.7668
Iteration No: 6 started. 

### Verificando quais parâmetros foram melhores

In [49]:
resultados_gp.x

[0.42925601142226866, 2, 11, 1.0, 0.7266023286040156]

## Criando o modelo 7 com os parâmetros acima

In [50]:
X_treino, y_treino = treino[features], treino[target]
X_teste = teste[features]


modelo7 = XGBClassifier(learning_rate=0.42925601142226866,
                         max_depth=2, 
                         min_child_weight=11, 
                         subsample=1.0, 
                         colsample_bytree=0.7266023286040156, 
                         random_state=0, 
                         n_estimators=100)

modelo7.fit(X_treino, y_treino)
y_pred7 = modelo7.predict(X_teste)

y_pred7 = y_pred7.astype(int)

y_pred7

array([0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

## Arquivo de submissão 7

In [51]:
get_submission(y_pred7, name_file='Setimo_modelo.csv')

# Modelo 8 - Rede Neural MultiLayer Perceptron

In [55]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score

In [58]:
treino[target].value_counts()    #tentar balancear essas classes

Survived
0.0    549
1.0    342
Name: count, dtype: int64

## Balanceando as classes com código do Let's Data

In [59]:
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

In [92]:
# Função disponível em:   https://github.com/letsdata/oversampling/blob/main/Classes%20Desbalanceadas.ipynb

# função de validação cruzada NA MÃO
def validacao_cruzada(modelo, X, y, oversampling=False):
    # KFold do scikit-learn
    kfold = KFold(n_splits=10)

    # lista de acuracias de cada split
    acuracias_split = []
    
    # iterando sobre os splits
    for idx, (idx_treino, idx_validacao) in enumerate(kfold.split(X)):
        X_split_treino = X.iloc[idx_treino, :]
        y_split_treino = y.iloc[idx_treino]
    
        # oversampling, só no split de treino!!
        if oversampling:
            sm = SMOTE(random_state=42)
            X_split_treino, y_split_treino = sm.fit_resample(X_split_treino, y_split_treino)
        
        # Com os dados balenceados SÓ NO TREINO, vamos treinar o nosso modelo
        modelo.fit(X_split_treino, y_split_treino.values.flatten())
    
        X_split_validacao = X.iloc[idx_validacao, :]
        y_split_validacao = y.iloc[idx_validacao]
        
        # Validação SEM oversampling, como a cartilha ensina :)
        # Amostra do mundo real, ou seja, com dados DESBALANCEADOS
        predicoes_validacao = modelo.predict(X_split_validacao)
        
        acuracia_split = accuracy_score(y_split_validacao, predicoes_validacao)
        
        acuracias_split.append(acuracia_split)
    
    return acuracias_split

### Bayesian Optimization antes

In [93]:
def treinar_mlp_acc(params):
    hidden1 = params[0]
    hidden2 = params[1]
    learning_rate = params[2]
    activation = params[3]
    solver = params[4]
    max_iter = params[5]
    tol = params[6]

    modelo = MLPClassifier(hidden_layer_sizes=(hidden1, hidden2),learning_rate=learning_rate, activation=activation, solver=solver, max_iter=max_iter, tol=tol, random_state=42)

    modelo.fit(X_treino, y_treino)
    pred = modelo.predict(X_valid)

    media_acc_smote = np.mean(validacao_cruzada(modelo, X_treino, y_treino, oversampling=True))

    return -media_acc_smote   #negativo pois a função de otimização é de minimização

In [94]:
from skopt.space import Categorical

# Definir espaço de busca
space = [
    (10, 50),   #hidden1
    (10, 50),    #hidden2
    Categorical(['constant', 'invscaling', 'adaptive'], name='learning_rate'),   #learning_rate
    Categorical(['identity', 'logistic', 'tanh', 'relu'], name='activation'),  #activation 
    Categorical(['adam', 'sgd'], name='solver'),  #solver
    (200, 1000), #max_iter
    (0.0001, 0.005)  #tol
    ]

In [95]:
resultados_gp = gp_minimize(treinar_mlp_acc, space, random_state=0, verbose=1, n_calls=50, n_random_starts=10)

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 9.1084
Function value obtained: -0.7532
Current minimum: -0.7532
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 2.3803
Function value obtained: -0.7655
Current minimum: -0.7655
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 3.6022
Function value obtained: -0.7789
Current minimum: -0.7789
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 7.8914
Function value obtained: -0.7296
Current minimum: -0.7789
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 3.6164
Function value obtained: -0.7789
Current minimum: -0.7789
Iteration No: 6 started. 

c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the 

Iteration No: 25 ended. Search finished for the next optimal point.
Time taken: 15.3513
Function value obtained: -0.7643
Current minimum: -0.7912
Iteration No: 26 started. Searching for the next optimal point.
Iteration No: 26 ended. Search finished for the next optimal point.
Time taken: 3.2990
Function value obtained: -0.7890
Current minimum: -0.7912
Iteration No: 27 started. Searching for the next optimal point.
Iteration No: 27 ended. Search finished for the next optimal point.
Time taken: 3.4797
Function value obtained: -0.7890
Current minimum: -0.7912
Iteration No: 28 started. Searching for the next optimal point.
Iteration No: 28 ended. Search finished for the next optimal point.
Time taken: 3.4351
Function value obtained: -0.7890
Current minimum: -0.7912
Iteration No: 29 started. Searching for the next optimal point.


c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (373) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (373) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (373) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (373) reached and the 

Iteration No: 29 ended. Search finished for the next optimal point.
Time taken: 33.7152
Function value obtained: -0.7755
Current minimum: -0.7912
Iteration No: 30 started. Searching for the next optimal point.
Iteration No: 30 ended. Search finished for the next optimal point.
Time taken: 4.1391
Function value obtained: -0.7890
Current minimum: -0.7912
Iteration No: 31 started. Searching for the next optimal point.
Iteration No: 31 ended. Search finished for the next optimal point.
Time taken: 2.4546
Function value obtained: -0.7845
Current minimum: -0.7912
Iteration No: 32 started. Searching for the next optimal point.
Iteration No: 32 ended. Search finished for the next optimal point.
Time taken: 5.9902
Function value obtained: -0.7834
Current minimum: -0.7912
Iteration No: 33 started. Searching for the next optimal point.
Iteration No: 33 ended. Search finished for the next optimal point.
Time taken: 3.5149
Function value obtained: -0.7800
Current minimum: -0.7912
Iteration No: 34 s

...

Iteration No: 36 ended. Search finished for the next optimal point.
Time taken: 41.8495
Function value obtained: -0.7789
Current minimum: -0.7912
Iteration No: 37 started. Searching for the next optimal point.
Iteration No: 37 ended. Search finished for the next optimal point.
Time taken: 4.0348
Function value obtained: -0.7890
Current minimum: -0.7912
Iteration No: 38 started. Searching for the next optimal point.
Iteration No: 38 ended. Search finished for the next optimal point.
Time taken: 2.8053
Function value obtained: -0.6803
Current minimum: -0.7912
Iteration No: 39 started. Searching for the next optimal point.
Iteration No: 39 ended. Search finished for the next optimal point.
Time taken: 4.3951
Function value obtained: -0.7845
Current minimum: -0.7912
Iteration No: 40 started. Searching for the next optimal point.
Iteration No: 40 ended. Search finished for the next optimal point.
Time taken: 3.2114
Function value obtained: -0.7476
Current minimum: -0.7912
Iteration No: 41 s

c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the 

Iteration No: 42 ended. Search finished for the next optimal point.
Time taken: 20.1501
Function value obtained: -0.7621
Current minimum: -0.7912
Iteration No: 43 started. Searching for the next optimal point.
Iteration No: 43 ended. Search finished for the next optimal point.
Time taken: 39.8049
Function value obtained: -0.7733
Current minimum: -0.7912
Iteration No: 44 started. Searching for the next optimal point.
Iteration No: 44 ended. Search finished for the next optimal point.
Time taken: 3.9242
Function value obtained: -0.7755
Current minimum: -0.7912
Iteration No: 45 started. Searching for the next optimal point.
Iteration No: 45 ended. Search finished for the next optimal point.
Time taken: 4.5111
Function value obtained: -0.7789
Current minimum: -0.7912
Iteration No: 46 started. Searching for the next optimal point.


c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (322) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (322) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (322) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (322) reached and the 

Iteration No: 46 ended. Search finished for the next optimal point.
Time taken: 28.7887
Function value obtained: -0.8081
Current minimum: -0.8081
Iteration No: 47 started. Searching for the next optimal point.
Iteration No: 47 ended. Search finished for the next optimal point.
Time taken: 36.6147
Function value obtained: -0.8047
Current minimum: -0.8081
Iteration No: 48 started. Searching for the next optimal point.
Iteration No: 48 ended. Search finished for the next optimal point.
Time taken: 29.2109
Function value obtained: -0.7969
Current minimum: -0.8081
Iteration No: 49 started. Searching for the next optimal point.
Iteration No: 49 ended. Search finished for the next optimal point.
Time taken: 4.2778
Function value obtained: -0.6825
Current minimum: -0.8081
Iteration No: 50 started. Searching for the next optimal point.
Iteration No: 50 ended. Search finished for the next optimal point.
Time taken: 40.9496
Function value obtained: -0.7879
Current minimum: -0.8081


In [96]:
resultados_gp.x

[10, 50, 'constant', 'tanh', 'adam', 322, 0.0001]

In [97]:
modelo8 = MLPClassifier(hidden_layer_sizes=(10, 50),
                        learning_rate='constant', 
                        activation='tanh', 
                        solver='adam', 
                        max_iter=322, 
                        tol=0.0001, 
                        random_state=42)

In [98]:
X_treino, y_treino = treino[features], treino[target]
X_teste = teste[features]

In [99]:
media_acc_com_smote = np.mean(validacao_cruzada(modelo8, X_treino, y_treino, oversampling=True)) * 100

c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (322) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (322) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (322) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (322) reached and the 

In [100]:
media_acc_com_smote

80.81023720349562

### Usar com SMOTE

In [101]:
validacao_cruzada(modelo8, X_treino, y_treino, oversampling=True)

c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (322) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (322) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (322) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (322) reached and the 

[0.7888888888888889,
 0.8089887640449438,
 0.7640449438202247,
 0.8202247191011236,
 0.8089887640449438,
 0.7865168539325843,
 0.797752808988764,
 0.797752808988764,
 0.8651685393258427,
 0.8426966292134831]

In [102]:
y_pred8 = modelo8.predict(X_teste)

y_pred8 = y_pred8.astype(int)

y_pred8

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

## Arquivo de submissão 8

In [103]:
get_submission(y_pred8, name_file='Oitavo_modelo.csv')

In [104]:
# Arquivo 8.2 SEM SMOTE

validacao_cruzada(modelo8, X_treino, y_treino, oversampling=False)

y_pred8_2 = modelo8.predict(X_teste)

y_pred8_2 = y_pred8_2.astype(int)

y_pred8_2

get_submission(y_pred8_2, name_file='Oitavo_modelo2.csv')